# import libraries

In [34]:
import geopandas as gpd
from shapely.geometry import Polygon
import json
import pandas as pd
file_path = "turkiye-ilceler.geojson"
# gdf_origin = gpd.read_file(file_path)

# print(gdf.head())


In [35]:
gdf = gdf_origin.copy()

In [36]:
gdf

,synonyms,icon,extrude,visibility,end,begin,timestamp,description,name,drawOrder,altitudeMode,tessellate,geometry
0,Çanakkale|Marmara|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Çan,NaN,NaN,-1,"POLYGON ((26.98408 39.86386, 26.95035 39.89492..."
1,Çanakkale|Marmara|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ayvacık (Çanakkale),NaN,NaN,-1,"POLYGON ((26.15903 39.64958, 26.19532 39.65125..."
2,Çanakkale|Marmara|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Bayramiç,NaN,NaN,-1,"POLYGON ((26.42260 39.69836, 26.42565 39.75752..."
3,Çanakkale|Marmara|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Biga,NaN,NaN,-1,"POLYGON ((27.57681 40.31189, 27.57907 40.27560..."
4,Çanakkale|Marmara|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Bozcaada,NaN,NaN,-1,"POLYGON ((26.05347 39.84736, 26.07014 39.84125..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Devrek,NaN,NaN,-1,"POLYGON ((32.18047 41.18181, 32.14834 41.16673..."
925,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ereğli (Zonguldak),NaN,NaN,-1,"POLYGON ((31.49785 41.05423, 31.47446 41.08943..."
926,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Gökçebey,NaN,NaN,-1,"POLYGON ((32.35404 41.23849, 32.29877 41.23664..."
927,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Merkez (Zonguldak),NaN,NaN,-1,"POLYGON ((32.06314 41.32787, 32.05437 41.29467..."


In [37]:
gdf[gdf['synonyms'].str.contains('\|Akdeniz\|')]

,synonyms,icon,extrude,visibility,end,begin,timestamp,description,name,drawOrder,altitudeMode,tessellate,geometry
38,Adana|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,İmamoğlu,NaN,NaN,-1,"POLYGON ((35.55980 37.21034, 35.54070 37.22733..."
39,Adana|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Aladağ,NaN,NaN,-1,"POLYGON ((35.43950 37.30673, 35.40385 37.33866..."
40,Adana|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ceyhan,NaN,NaN,-1,"POLYGON ((35.93405 36.87986, 35.91014 36.85797..."
41,Adana|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Feke,NaN,NaN,-1,"POLYGON ((35.64175 37.70995, 35.63138 37.72034..."
42,Adana|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Karaisalı,NaN,NaN,-1,"POLYGON ((35.38538 37.22118, 35.36890 37.19987..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,Osmaniye|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Hasanbeyli,NaN,NaN,-1,"POLYGON ((36.50293 37.05405, 36.46766 37.05895..."
742,Osmaniye|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Kadirli,NaN,NaN,-1,"POLYGON ((36.29478 37.35043, 36.26087 37.30351..."
743,Osmaniye|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Merkez (Osmaniye),NaN,NaN,-1,"POLYGON ((36.31174 36.96848, 36.28161 36.99868..."
744,Osmaniye|Akdeniz|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Sumbas,NaN,NaN,-1,"POLYGON ((35.88216 37.26752, 35.88317 37.35688..."


In [42]:
gdf['synonyms'] = gdf['synonyms'].str.replace(r'\|[^|]*\|','|',regex=True)

In [45]:
gdf

,synonyms,icon,extrude,visibility,end,begin,timestamp,description,name,drawOrder,altitudeMode,tessellate,geometry
0,Çanakkale|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Çan,NaN,NaN,-1,"POLYGON ((26.98408 39.86386, 26.95035 39.89492..."
1,Çanakkale|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ayvacık (Çanakkale),NaN,NaN,-1,"POLYGON ((26.15903 39.64958, 26.19532 39.65125..."
2,Çanakkale|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Bayramiç,NaN,NaN,-1,"POLYGON ((26.42260 39.69836, 26.42565 39.75752..."
3,Çanakkale|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Biga,NaN,NaN,-1,"POLYGON ((27.57681 40.31189, 27.57907 40.27560..."
4,Çanakkale|Marmara Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Bozcaada,NaN,NaN,-1,"POLYGON ((26.05347 39.84736, 26.07014 39.84125..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Devrek,NaN,NaN,-1,"POLYGON ((32.18047 41.18181, 32.14834 41.16673..."
925,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ereğli (Zonguldak),NaN,NaN,-1,"POLYGON ((31.49785 41.05423, 31.47446 41.08943..."
926,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Gökçebey,NaN,NaN,-1,"POLYGON ((32.35404 41.23849, 32.29877 41.23664..."
927,Zonguldak|Karadeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Merkez (Zonguldak),NaN,NaN,-1,"POLYGON ((32.06314 41.32787, 32.05437 41.29467..."


In [43]:
gdf[gdf['synonyms'].str.contains('Akdeniz')]

,synonyms,icon,extrude,visibility,end,begin,timestamp,description,name,drawOrder,altitudeMode,tessellate,geometry
38,Adana|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,İmamoğlu,NaN,NaN,-1,"POLYGON ((35.55980 37.21034, 35.54070 37.22733..."
39,Adana|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Aladağ,NaN,NaN,-1,"POLYGON ((35.43950 37.30673, 35.40385 37.33866..."
40,Adana|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Ceyhan,NaN,NaN,-1,"POLYGON ((35.93405 36.87986, 35.91014 36.85797..."
41,Adana|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Feke,NaN,NaN,-1,"POLYGON ((35.64175 37.70995, 35.63138 37.72034..."
42,Adana|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Karaisalı,NaN,NaN,-1,"POLYGON ((35.38538 37.22118, 35.36890 37.19987..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,Osmaniye|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Hasanbeyli,NaN,NaN,-1,"POLYGON ((36.50293 37.05405, 36.46766 37.05895..."
742,Osmaniye|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Kadirli,NaN,NaN,-1,"POLYGON ((36.29478 37.35043, 36.26087 37.30351..."
743,Osmaniye|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Merkez (Osmaniye),NaN,NaN,-1,"POLYGON ((36.31174 36.96848, 36.28161 36.99868..."
744,Osmaniye|Akdeniz Bölgesi,NaN,0,1,NaN,NaN,NaN,District,Sumbas,NaN,NaN,-1,"POLYGON ((35.88216 37.26752, 35.88317 37.35688..."


# export file

In [46]:
output_file = 'turkiye-ilceler-bolgesiz.geojson'
gdf.to_file(output_file, driver="GeoJSON")